# Implementing Long-Term Memory in LangGraph

> **Learning Outcomes:**
> - Understand the concept and mechanisms of Long-Term Memory
> - Implement Long-Term Memory in LangGraph
> - Extend the implementation to support consolidation

## Introduction

In this lab, we will implement long-term memory in LangGraph. Long-Term memory is a mechanism that allows the model to remember information from the past. This is particularly useful in tasks that require the model to remember information from the past interactions to tailor their responses to the user.

### Memory in Humans

In humans, memory has many different components and processes. These include:

1. *Encoding:* Converting information into a form that can be stored in memory.
2. *Storage:* Storing information in memory.
3. *Recall:* Passively remembering information.
4. *Retrieval:* Actively trying to remember information.
5. *Forgetting:* The loss of information over time.
6. *Consolidation:* Stabilizing and organizing memories over time.

### The Scenario

We will be creating a chatbot that can remember information from past interactions. The chatbot will be able to selectively remember information that is important and forget information that is not. The chatbot will also be able to consolidate memories to make them more stable and organized.

The graph will look like this:

```mermaid
stateDiagram
    [*] --> Recall
    Recall --> ChatBot
    ChatBot --> Encode
    Encode --> Consolidate
    Consolidate --> Chat
```

## Getting Started

Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.

In [ ]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

In [ ]:
%pip install -qU langchain==0.3.* langchain_openai==0.3.* langgraph==0.5.*

import os
import getpass
import uuid
import textwrap
from typing import Annotated

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")

## Core

### Step 1 - Building the State

We will start by building the state of our graph. The state will contain the messages from the user and the chatbot. The state will also contain retrieved memories from the long-term memory.

We will use the `TypedDict` class from the `typing_extensions` module to define the state. This will allow us to specify the types of the state variables. We will also use the `Annotated` class to add metadata to the state variables. This includes the "add_messages" metadata for the `messages` field, which specifies that new messages should be added to the existing messages.

```python
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
```

In the follow cell, define a `ChatbotState` class with the following fields:

- `messages`: A list of messages from the user and the chatbot. 
- `memories`: A list of strings representing the memories stored in the long-term memory.

```python

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph import add_messages

# YOUR CODE HERE


This is how LangGraph will pass data between nodes in the graph. As the data moves through the graph, it will be stored in a typed dictionary. Fields can be added or updated as the data moves through the graph.

### Step 2 - Setting Up the Memory Store

Next, we will set up the memory store for the long-term memory. The memory store will use LangGraph's `InMemoryStore`. This memory doesn't refer to "memory", but to RAM. So, it will reset and lose all data when the program is restarted.

We will also create a `Memories` class that will allow us to structure the outputs returned from a model and make it easier to work with the memories.

Run the following cell to define both.

In [ ]:
from langgraph.store.memory import InMemoryStore
from langchain_openai import OpenAIEmbeddings
from pydantic import BaseModel

in_memory_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "dims": 1536,
    }
)

class Memories(BaseModel):
    list: list[str]


### Step 3 - Implementing the Nodes

Now, we will implement the nodes for our graph. To make working with our memory store easier, we will first define a helper function named `get_namespace` that will return a namespace for the memory store.

Run the following cell to define the `get_namespace` function.

In [ ]:
from langchain_core.runnables import RunnableConfig

def get_namespace(config: RunnableConfig):
    return ("memories", config["configurable"]["user_id"])

#### Recall Node

Our first node will be the `Recall` node. This node will retrieve memories from the long-term memory store.

You can retrieve memories from the memory store using the `search` method of the memory store. The `get` method takes the namespace and a query as arguments. The query can be a string or a dictionary.

```python
results = store.search(namespace, query)
results[0].value["data"] # <-- This will give you the memory
```

To simulate passive recall of memories, we will use the user's last message as the query to retrieve memories related to the user's last message.

```python
last_message = state["messages"][-1].content
```

In the following cell, implement the `Recall` node. The node should retrieve memories related to the user's last message and update the memroies in the state.

In [ ]:
from langgraph.store.base import BaseStore

def recall(state: ChatbotState, config: RunnableConfig, *, store: BaseStore):
    # YOUR CODE HERE


#### Chatbot Node

Our next node will be the `ChatBot` node. This node will generate a response from the chatbot.

In the following cell, write a system message that describes the purpose of the chatbot and add the memories to it. Then invoke the model with the system message and the state messages.

Here is a brief example that does not include the memories:

```python
model.invoke(
    [{"role": "system", "content": "I am a chatbot that can remember information from past interactions."}]
    + state["messages"]
)
```

Make sure to return the response to add it to the messages!

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4.1")

def chatbot(state: ChatbotState, config: RunnableConfig, *, store: BaseStore):
    # YOUR CODE HERE


#### Encode Node
Next, we will implement the `Encode` node. This node will encode the chatbot's response and store it in the long-term memory.

This node will function similarly to the chatbot mode. However, it should have an additional user message that asks the model to make a list of things to remember. It should return the `Memories` object with the memories to store. To store the memories, you can use the the following code:

```python
store.put(namespace, str(uuid.uuid4()), {"data": memory})
```

In the following cell, implement the `Encode` node.

In [ ]:
def encode(state: ChatbotState, config: RunnableConfig, *, store: BaseStore):
    # Pre-format the memories string to avoid f-string backslash error
    existing_memories_str = "\n".join(state["memory"])

    memories = model.with_structured_output(Memories).invoke([
        {
            "role": "system",
            "content": textwrap.dedent("""
                You are a bag person for the assistant.
                Help the assistant remember important information.
                Format memories as declarative sentence fragments.
                Do not include already known info.
            """).strip()
        }
    ] + state["messages"][-2:] + [
        {
            "role": "user",
            "content": textwrap.dedent(f"""
                # Memories
                {existing_memories_str}

                Make a list of things to remember.
            """)
        }
    ])

    namespace = get_namespace(config)

    for memory in memories.list:
        print("memory:", memory)
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

#### Consolidate Node

Our final node will be the `Consolidate` node. This node will consolidate the memories in the long-term memory store. We are going to use the internals of the memory store to make it easier to retrieve and delete all the memories.

```python
store._data[namespace] # <-- This will give you all the memories
del store._data[namespace] # <-- This will delete all the memories
```

In the cell below, implement the `Consolidate` node. The node should consolidate the memories in the long-term memory store and store the consolidated memories in the state.

In order to prevent this node from running every time, add a condition to run this node only if there are more than 10 memories in the memory store. Limit the number of memories to 10.

In [ ]:
def consolidate(state: ChatbotState, config: RunnableConfig, *, store: BaseStore):
    namespace = get_namespace(config)
    all_memories = store._data[namespace]

    # YOUR CODE HERE


### Step 4 - Building the Graph

Now we can build the graph using the nodes we implemented. The graph is fairly straightforward and links the nodes in the following order: Recall -> ChatBot -> Encode -> Consolidate. As a reminder, this is how you can build the graph:

```python
builder = StateGraph()
builder.add_node("node", function)
builder.add_edge(START, "node")
builder.add_edge("node", END)
```

In the cell below, build the graph using the nodes we implemented. At the end, compile it using the following code:

```python
builder.compile(checkpointer=MemorySaver(), store=in_memory_store)
```

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(ChatbotState)
# YOUR CODE HERE


### Step 5 - Running the Graph

Finally, we can run the graph. Run the following cell to run the graph and see the chatbot in action!

In [ ]:
# Set a user ID and thread ID for the conversation. (Thread is the conversation, not the code thread.)
thread_1 = {"configurable": {"thread_id": "1", "user_id": "1"}}
thread_2 = {"configurable": {"thread_id": "2", "user_id": "1"}}

# Start the conversation with a message from the user.
input_message = {
    "role": "user",
    "content": "Hi! I'm Bob."
}

def run_graph(input_message, config):
    for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
        final_state = chunk
        print(chunk)

    chunk["messages"][-1].pretty_print()

run_graph(input_message, thread_1)

Now let's give the model a lot to remember!

In [ ]:
run_graph({"role": "user", "content": """
I prefer tea over coffee.
I have a small scar on my left elbow.
I enjoy solving crossword puzzles.
I am afraid of heights.
I can play the first few bars of "Für Elise" on piano.
I own a collection of vintage postcards.
I have traveled to three different continents.
I am a night owl.
I love the smell of old books.
I once won a local pie-baking contest.

What should I bake this weekend?
"""}, thread_1)

Run the cell below to see if the chatbot can remember the information across conversations.

In [ ]:
run_graph({"role": "user", "content": """
What do you remember about me?
"""}, thread_2)

## Bonus Challenge
Feel free to experiment with the chatbot by sending different messages. You can also modify the code to add more features or improve the existing ones.

In [ ]:
# YOUR CODE 

## Conclusion
In this lab, we implemented long-term memory in LangGraph. We created a chatbot that can remember information from past interactions and consolidate memories to make them more stable and organized. We also learned how to use the memory store to store and retrieve memories.